## LDA(Latent Dirichlet Allocation): scikit-learn

### 데이터 준비

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 패키지 업그레이드로 인한 에러 처리를 위해 라이브러리 버전 Downgrade
!pip install 'networkx<2.7'


In [ ]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
import pandas as pd

In [3]:
news = pd.read_csv('뉴스s.csv') 

In [67]:
def datacleansing (data):
    new_df = data
    
    #뉴스기사 출처 ( = ) 형태 없애기 코드 '\(.*=.*\) '
    new_df['content']=new_df['content'].map(lambda x:re.sub('\(.*=.*\) ', '', str(x))) 
    #뉴스기사 출처 ( = ) 형태 없애기 코드 '\[.*=.*\] '
    new_df['content']=new_df['content'].map(lambda x:re.sub('\[.*=.*\] ', '', str(x))) 
    
    # 날짜 지우기 '2023.[0-9]?[0-9].[0-9]?[0-9]'
    new_df['content']=new_df['content'].map(lambda x:re.sub('2023.[0-9]?[0-9].[0-9]?[0-9]', '', str(x))) 
    
    # '▶제보는 카톡 okjebo' 없애기'
    new_df['content']=new_df['content'].map(lambda x:re.sub('▶제보는 카톡 okjebo' , '', str(x))) 
    
    # '재판매 및 DB 금지'  없애기
    new_df['content']=new_df['content'].map(lambda x:re.sub('재판매 및 DB 금지', '', str(x))) 
    
    # '※ 자세한 내용은 동영상으로 확인하실 수 있습니다.'  없애기
    new_df['content']=new_df['content'].map(lambda x:re.sub('※ 자세한 내용은 동영상으로 확인하실 수 있습니다.', '', str(x))) 
    
    #※CBS노컷뉴스는 여러분의 제보로 함께 세상을 바꿉니다. 각종 비리와 부당대우, 사건사고와 미담 등 모든 얘깃거리를 알려주세요.\n이메일 :\n카카오톡 :@노컷뉴스\n사이트 :\nCBS노컷뉴스 김승모 기자 \n기자와 카톡 채팅하기 노컷뉴스 영상 구독하기'
    sth = '※CBS노컷뉴스는 여러분의 제보로 함께 세상을 바꿉니다. 각종 비리와 부당대우, 사건사고와 미담 등 모든 얘깃거리를 알려주세요.\n이메일 :\n카카오톡 :@노컷뉴스\n사이트 :\nCBS노컷뉴스 김승모 기자 \n기자와 카톡 채팅하기 노컷뉴스 영상 구독하기'
    new_df['content']=new_df['content'].map(lambda x:re.sub(sth, '', str(x))) 
    #무단전재 및 재배포 금지 없애기기
    new_df['content']=new_df['content'].map(lambda x:re.sub('무단전재 및 재배포 금지', '', str(x))) 
    # 제공 없애기
    new_df['content'] = new_df['content'].map(lambda x:re.sub(' 제공', '', str(x))) 

    return new_df

In [68]:
news = datacleansing(news)

In [70]:
news['sub_category'].unique()

array(['IT기기', 'Tech일반', '게임', '소프트웨어', '인터넷', '통신_모바일', '휴대폰통신'],
      dtype=object)

In [71]:
news.drop(news[news['sub_category'] == '과학'].index, inplace=True)

In [72]:
news['sub_category'].unique()

array(['IT기기', 'Tech일반', '게임', '소프트웨어', '인터넷', '통신_모바일', '휴대폰통신'],
      dtype=object)

In [73]:
news.dropna(subset=['content'], inplace=True) #content null 값 행 삭제 

In [74]:
news.drop_duplicates(subset=['content'], ignore_index=True, inplace=True) # 중복 기사 삭제 

### 전처리 필요 부분 확인

In [ ]:
news['content'].iloc[0]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 전담할 '우주항공청'을 출범시킬 계획이다. 외부 전문가 채용 비중 제한을 없애고, 우수 인력을 유치하고자 주식백지신탁 예외까지 허용하는 등 우주항공청의 전문성을 높이기 위한 다양한 특례까지 마련된다. 과학기술정보통신부는 우주항공청 설립의 근거가 되는 '우주항공청의 설치 및 운영에 관한 특별법' 제정안을 2일부터 입법예고한다고 밝혔다. 특별법은 우주항공분야의 정책과 R&D, 산업육성 등을 총괄하는 중앙행정기관으로 우주항공청을 설치하고, 전문적이고 유연한 조직으로 운영하기 위한 원칙과 기능, 특례 등까지 명시하고 있다. 먼저 우주항공 정책 거버넌스를 구축하기 위해 과기정통부, 산업통상자원부 등의 부처가 개별적으로 수행하던 우주항공 관련 기술개발, 산업육성 지원, 인재양성 및 우주위험 대비 등의 기능을 모두 우주항공청으로 일원화했다. 이를 위해 기존 과기정통부 장관, 산지부 장관 등이 소관하던 우주항공 관련 법률인 '우주개발진흥법', '항공우주산업촉진법', '천문법' 등을 우주항공청장이 담당하도록 부칙을 통해 개정할 방침이다. 국가우주위원회 위원장은 국무총리에서 대통령으로 격상해 위상과 기능을 강화한다. 우주항공청장도 국가우주위에 새롭게 위원으로 추가해 실무위원회 위원장을 맡게 하는 등 우주항공청이 우주경제 시대를 여는 컨트롤 타워 역할을 하도록 했다. 우주항공청 조직의 경우 과기정통부 장관 소속으로 하되 전문성을 가진 조직으로 구성된다. 특히 우주항공청 조직 구성 원칙의 핵심은 유연하고 자율적인 운영이다. 이를 위해 우주항공청 내에 설치되는 별도의 본부가 우주항공 R&D와 산업 활성화를 전담하게 된다."

In [ ]:
# ▲ 없애기 
text = re.sub(r'[▲]','', text)

#뉴스기사 출처 ( = ), [ = ] 형태 없애기 코드 '\(.*=.*\) ' 
text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))

#[] 안에 들어가는 거 다 없애기? -> 그러면 안됨.. 

# 괄호 안 기자로 끝나는 거 지우기 
#re.sub(r'\[.*기자\]', '', text)

text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)

 # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)

In [ ]:
a = news['content'].str.contains('페이스북 /LeYN1 트위터 @yonhap_graphics')

print((news['content'].loc[a]))

7528     \n ,(서울=연합뉴스) 원형민 기자 = 정부가 미래산업의 성장동력을 확보하기 위한...
9133     \n ,(서울=연합뉴스) 원형민 기자 = 한국은행이 22일 발표한 '2월 기업경기실...
9937     \n ,(서울=연합뉴스) 원형민 기자 = 서방이 우크라이나 침공에 대한 제재로 러시...
16843    \n ,(서울=연합뉴스) 반종빈 기자 = 일본은 16일 한국에 대한 반도체 핵심 소...
Name: content, dtype: object


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
news['content'].iloc[9133]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\n ,(서울=연합뉴스) 원형민 기자 = 한국은행이 22일 발표한 '2월 기업경기실사지수(BSI)' 조사 결과에 따르면 이달 제조업 업황 BSI는 전월보다 3포인트 하락한 63을 기록했다., , 페이스북 /LeYN1 트위터 @yonhap_graphics,\n "

### tokenization

In [13]:
!pip3 install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    text = re.sub(r'[▲]','', text) 
    text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))
    text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)

    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return, 단어 간 관계 파악 위함으로 설정 안함 
    
    words = []
    stopwords = ['하는', '있다', '있는', '위해', '통해', '한다', '때문', '했다'] # 불용어 사전  
    for word, pos in Okt_morphs:
        if pos in ['Adjective', 'Verb', 'Noun', 'Alpha', 'Number'] and word not in stopwords:  # 이 경우에는 형용사, 동사, 명사, 영어, 숫자만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# news['content']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in news['content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

532
월간 자동차 는 월별 판매 데이터 기준 자동차 소비 트렌드 분석 월 1회 연재 코너 입니다 브랜드 별 차량 별 인기 순위 사용 연료 사용자 중심 IT 널 IT 동아


In [15]:
#단어가 1-20개만 포함된 corpus 삭제 
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus.split())) < 21:   # 같은 단어 1-20개만 반복되는 corpus도 지우기 위해 set()을 사용
        news.drop(index=[index], axis=0, inplace=True)
        drop_corpus.append(corpus)
                
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

news.reset_index(drop=True, inplace=True)

print(len(tokenized_list))
print(len(news))

532
532


### vectorization & LDA

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [17]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(tokenized_list)
word_count = X.toarray().sum(axis=1)
print("Total number of tokens:", word_count.sum())

Total number of tokens: 102238


In [18]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(tokenized_list)
unique_tokens = count_vectorizer.get_feature_names_out()
count_unique_tokens = len(unique_tokens)
print("Total number of unique tokens:", len(unique_tokens))

Total number of unique tokens: 12437


#### vectorization

In [19]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer_0 = CountVectorizer(max_df=0.8, max_features=round(count_unique_tokens*0.8), min_df=10, ngram_range=(1,2))
    # 10개의 문서 미만으로 등장하는 단어는 제외, 전체의 80% 이상으로 자주 등장하는 단어는 제외, 최대로 사용할 단어 수(백터 수)는 유니크한 토큰 수의 80% 
    # unigram, bigram 포함

feat_vect = count_vectorizer_0.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape) #문서의 수, 단어의 수(=유니크한 토큰 수)

CountVectorizer Shape: (532, 2041)


#### topic modeling: LDA

In [20]:
lda = LatentDirichletAllocation(n_components=7)  # 토픽 수는 7개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=7)

#### 토픽별 연관어 출력

In [21]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names_out()를 통해 추출
feature_names = count_vectorizer_0.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
kt 사외이사 후보 대표 고문 선임 이사회 아이 뉴스 아이 뉴스
Topic # 1
사전 서버 진행 게임 예약 스포츠 이벤트 크리에이터 이스포츠 사전 예약
Topic # 2
kt 알뜰폰 lg 현대차 lg 전자 대표 전자 통신 의혹 사장
Topic # 3
게임 이용자 카카오 제공 플레이 진행 출시 다양한 서비스 콘텐츠
Topic # 4
게임 네이버 개발 글로벌 넥슨 출시 매출 프로젝트 지난 크로니클
Topic # 5
ai gpt 서비스 기능 오픈 클라우드 사용 오픈 ai 애저 기술
Topic # 6
기업 기술 데이터 서비스 시장 사업 국내 제공 지난해 개발


### LDA 시각화

In [ ]:
!pip install pyLDAvis # pyLDAvis 라이브러리를 이용한 시각화 

In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer.get_feature_names_out())
pyLDAvis.display(vis) 

In [ ]:
pyLDAvis.show(vis) 

### 문서별 토픽 할당

#### 1. 각 문서별로 가장 가까운 topic으로 할당

In [22]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌 
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,6,0.821098
1,1,6,0.636965
2,2,6,0.714030
3,3,6,0.994757
4,4,6,0.515956


In [23]:
doc_topic_df = doc_topic_df.join(news)
doc_topic_df.head() #실제 뉴스기사와 join

,Doc_Num,Topic,Percentage,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,0,6,0.821098,595232,"[월간 자동차] 2023년 2월, 국산 신차 등록률 하락…회복세 보이는 중고차 시장",김동진,'월간 자동차'는 월별 판매 데이터를 기준으로 자동차 소비 트렌드를 분석하는 월 1...,2023-03-10 19:06:00,https://v.daum.net/v/20230310190605071,IT,IT기기,다음
1,1,6,0.636965,595233,같은 메뉴도 매장보다 배달앱에서 더 비싸… 배달가격에 기만당하는 소비자,정연호,소비자에겐 물건을 살 때 업주로부터 제품 정보를 충분히 제공받을 '알 권리'와 이...,2023-03-10 18:54:00,https://v.daum.net/v/20230310185403810,IT,IT기기,다음
2,2,6,0.714030,595234,아이폰15 조기 출하 소식에 LG이노텍 '방긋',류은주,아이폰15의 조기 출하설과 부품 비용 인상설이 맞물리며 LG이노텍을 비롯한 부품사의...,2023-03-10 16:58:00,https://v.daum.net/v/20230310165827089,IT,IT기기,다음
3,3,6,0.994757,595235,"삼성전자, 1분기 스마트폰 시장 1위 탈환",이나리,삼성전자가 지난해 4분기 스마트폰 시장에서 2위를 기록한데 이어 올 1분기에 1위로...,2023-03-10 16:29:00,https://v.daum.net/v/20230310162920365,IT,IT기기,다음
4,4,6,0.515956,595236,"[스타트업-ing] “대체육 다음은 대체 생선”…식물성 고등어, 출시 전 막바지 담금질",권택경,"""지금 당장 출시해도 손색 없는 수준입니다"" 지난 7일 서울 강동구 서울먹거리창업...",2023-03-10 16:18:00,https://v.daum.net/v/20230310161806102,IT,IT기기,다음


#### 2. 토픽별 문서 수 계산

In [24]:
doc_topic_df.groupby('Topic')[['Doc_Num']].count()

,Doc_Num
Topic,
0,64
1,49
2,39
3,82
4,57
5,62
6,179


#### 3. 토픽별로, 가장 높은 확률로 할당된 문서 top3 확인(이걸로 메인 뉴스 뽑기)

In [25]:
for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['content'].iloc[0])
    print(top_pr_topics['content'].iloc[1])
    print(top_pr_topics['content'].iloc[2], '\n')

Topic # 0 -----------------------------
[사진=넥슨] 넥슨(대표 이정헌)은 서브 브랜드 '민트로켓'의 첫 신작 '데이브 더 다이버(이하 데이브)'의 개발자 영상을 공개하고 올해 6월 정식 출시를 예고했다. 개발자 영상에는 황재호 디렉터와 우찬희 기획리더가 출연해 정식 출시 일정과 정식 버전 콘텐츠, 3월 업데이트 소식을 전했다. 데이브는 다가오는 6월 스팀을 통해 PC 버전을 정식 출시하며, 이후 다양한 플랫폼에서도 정식 서비스를 추가할 계획이다. 정식 버전에서는 '어인족 마을'에서 '벨루가'를 타고 쉽게 이동할 수 있으며 어두운 분위기의 새로운 지역 '빙하 통로'를 만나볼 수 있다. 또한 'VIP' 캐릭터와의 요리대결, 각종 미니게임, 캐릭터 스토리 미션, 신규 어종이 새롭게 추가된다. 3월 14일에는 콘텐츠 추가, 개선 업데이트를 실시한다. 맥 운영체제에서 게임 플레이가 가능하고 독일어, 스페인어, 중국어(번체)도 추가로 지원한다. '포토스팟' 재촬영 기능 추가, '양식장' 밀집도 개선, 아이템 선택적 획득 등 다양한 콘텐츠들을 개선했다. 넥슨은 봄을 맞아 3월 17일부터 24일까지 스팀 할인 행사를 실시한다. 기존 얼리 엑세스 가격에서 20% 할인한 1만9천200원에 데이브를 구매할 수 있다. 이후 공식 커뮤니티를 통해 팬아트 이벤트도 진행할 예정이다. 한편 데이브는 블루홀을 탐사하며 해양 생물을 포획하는 어드벤처와 포획한 해양 생물로 초밥집을 운영하는 타이쿤이 결합된 하이브리드 해양 어드벤처 게임이다. 지난 10월 스팀에서 얼리 액세스를 진행하며 글로벌 이용자들의 압도적 긍정 평가를 이어오고 있다. /문영수 기자() 네이버 채널에서 '아이뉴스24'를 구독해주세요. 재밌는 아이뉴스TV 영상보기아이뉴스24 바로가기 [ 아이뉴스24 무단전재 및 재배포 금지]
넥슨 서브 브랜드 민트로켓의 '데이브 더 다이버'가 오는 6월 정식 출시를 예고했다. /사진=넥슨 제공 넥슨코리아(대표 이정헌)가 게임 서브 브랜드 '민트로켓'(MINTR

#### 키워드 요약, 문장 요약을 위해 토픽 7개만 들어있는 리스트 만들기

In [26]:
#content만 담겨있는 list
top7_topic_contents = []

for topic in range(len(doc_topic_df['Topic'].unique())):
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    content1 = top_pr_topics['content'].iloc[0]
    top7_topic_contents.append([content1])

print(top7_topic_contents) 

[["[사진=넥슨] 넥슨(대표 이정헌)은 서브 브랜드 '민트로켓'의 첫 신작 '데이브 더 다이버(이하 데이브)'의 개발자 영상을 공개하고 올해 6월 정식 출시를 예고했다. 개발자 영상에는 황재호 디렉터와 우찬희 기획리더가 출연해 정식 출시 일정과 정식 버전 콘텐츠, 3월 업데이트 소식을 전했다. 데이브는 다가오는 6월 스팀을 통해 PC 버전을 정식 출시하며, 이후 다양한 플랫폼에서도 정식 서비스를 추가할 계획이다. 정식 버전에서는 '어인족 마을'에서 '벨루가'를 타고 쉽게 이동할 수 있으며 어두운 분위기의 새로운 지역 '빙하 통로'를 만나볼 수 있다. 또한 'VIP' 캐릭터와의 요리대결, 각종 미니게임, 캐릭터 스토리 미션, 신규 어종이 새롭게 추가된다. 3월 14일에는 콘텐츠 추가, 개선 업데이트를 실시한다. 맥 운영체제에서 게임 플레이가 가능하고 독일어, 스페인어, 중국어(번체)도 추가로 지원한다. '포토스팟' 재촬영 기능 추가, '양식장' 밀집도 개선, 아이템 선택적 획득 등 다양한 콘텐츠들을 개선했다. 넥슨은 봄을 맞아 3월 17일부터 24일까지 스팀 할인 행사를 실시한다. 기존 얼리 엑세스 가격에서 20% 할인한 1만9천200원에 데이브를 구매할 수 있다. 이후 공식 커뮤니티를 통해 팬아트 이벤트도 진행할 예정이다. 한편 데이브는 블루홀을 탐사하며 해양 생물을 포획하는 어드벤처와 포획한 해양 생물로 초밥집을 운영하는 타이쿤이 결합된 하이브리드 해양 어드벤처 게임이다. 지난 10월 스팀에서 얼리 액세스를 진행하며 글로벌 이용자들의 압도적 긍정 평가를 이어오고 있다. /문영수 기자() 네이버 채널에서 '아이뉴스24'를 구독해주세요. 재밌는 아이뉴스TV 영상보기아이뉴스24 바로가기 [ 아이뉴스24 무단전재 및 재배포 금지]"], ["예약자 수 200만 돌파한 '아키에이지 워'. 카카오게임즈 카카오게임즈는 엑스엘게임즈가 개발 중인 다중접속역할수행게임(MMORPG) '아키에이지 워'의 사전 예약자 수가 200만명을 달성했다고 10일 밝혔다. 아키에이지 워는 1

In [27]:
#news_id, title, content만 담겨있는 list
top7_topic = []

for topic in range(len(doc_topic_df['Topic'].unique())):
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    content1 = top_pr_topics.iloc[0]
    top7_topic.append([content1['news_id'], content1['title'], content1['content']])

print(top7_topic)

[[612247, "민트로켓, 해양 어드벤처 '데이브 더 다이버' 6월 정식 출시", "[사진=넥슨] 넥슨(대표 이정헌)은 서브 브랜드 '민트로켓'의 첫 신작 '데이브 더 다이버(이하 데이브)'의 개발자 영상을 공개하고 올해 6월 정식 출시를 예고했다. 개발자 영상에는 황재호 디렉터와 우찬희 기획리더가 출연해 정식 출시 일정과 정식 버전 콘텐츠, 3월 업데이트 소식을 전했다. 데이브는 다가오는 6월 스팀을 통해 PC 버전을 정식 출시하며, 이후 다양한 플랫폼에서도 정식 서비스를 추가할 계획이다. 정식 버전에서는 '어인족 마을'에서 '벨루가'를 타고 쉽게 이동할 수 있으며 어두운 분위기의 새로운 지역 '빙하 통로'를 만나볼 수 있다. 또한 'VIP' 캐릭터와의 요리대결, 각종 미니게임, 캐릭터 스토리 미션, 신규 어종이 새롭게 추가된다. 3월 14일에는 콘텐츠 추가, 개선 업데이트를 실시한다. 맥 운영체제에서 게임 플레이가 가능하고 독일어, 스페인어, 중국어(번체)도 추가로 지원한다. '포토스팟' 재촬영 기능 추가, '양식장' 밀집도 개선, 아이템 선택적 획득 등 다양한 콘텐츠들을 개선했다. 넥슨은 봄을 맞아 3월 17일부터 24일까지 스팀 할인 행사를 실시한다. 기존 얼리 엑세스 가격에서 20% 할인한 1만9천200원에 데이브를 구매할 수 있다. 이후 공식 커뮤니티를 통해 팬아트 이벤트도 진행할 예정이다. 한편 데이브는 블루홀을 탐사하며 해양 생물을 포획하는 어드벤처와 포획한 해양 생물로 초밥집을 운영하는 타이쿤이 결합된 하이브리드 해양 어드벤처 게임이다. 지난 10월 스팀에서 얼리 액세스를 진행하며 글로벌 이용자들의 압도적 긍정 평가를 이어오고 있다. /문영수 기자() 네이버 채널에서 '아이뉴스24'를 구독해주세요. 재밌는 아이뉴스TV 영상보기아이뉴스24 바로가기 [ 아이뉴스24 무단전재 및 재배포 금지]"], [612275, '카카오게임즈, ‘아키에이지 워’ 사전 예약자 200만 돌파…21일 출시', "예약자 수 200만 돌파한 '아키에이지 워'. 카카오게

In [46]:
top7_topic = pd.DataFrame(top7_topic, columns=['news_id', 'title', 'content','keywords','summary'])


In [47]:
top7_topic

,news_id,title,content,keywords,summary
0,612247,"민트로켓, 해양 어드벤처 '데이브 더 다이버' 6월 정식 출시",[사진=넥슨] 넥슨(대표 이정헌)은 서브 브랜드 '민트로켓'의 첫 신작 '데이브 더...,NaN,NaN
1,612275,"카카오게임즈, ‘아키에이지 워’ 사전 예약자 200만 돌파…21일 출시",예약자 수 200만 돌파한 '아키에이지 워'. 카카오게임즈 카카오게임즈는 엑스엘게임...,NaN,NaN
2,625128,"KT ""윤경림 사장 재입사, 현대차의 에어플러그 인수와 무관""","기사내용 요약 시민단체, 윤경림·구현모 관련 의혹 檢 고발건 의혹에 KT 반박 ""구...",NaN,NaN
3,606937,"카카오메이커스, 누적거래 7000억원 돌파… 6000억 넘은지 7개월만",카카오의 임팩트 커머스 '카카오메이커스'가 지난해 8월 누적 거래액 6000억원을 ...,NaN,NaN
4,612227,"글로벌서도 통한다… ‘크로니클’, 6개 국 애플 앱스토어 인기 1위",컴투스 제공. 컴투스 '서머너즈 워: 크로니클(이하 크로니클)'이 글로벌 시장에서도...,NaN,NaN
5,606899,"챗GPT로 뽕을 뽑는 MS, 검색엔진·클라우드·협업툴까지 고도화",마이크로소프트 CI MS(마이크로소프트)가 자사의 클라우드 플랫폼 애저(Azure)...,NaN,NaN
6,606879,하이브리드 클라우드 데이터 최적화 기술 ‘IBM LTO 9 테이프’,GII(Global Information) 시장보고서에 따르면 전세계 테이프 스토리...,NaN,NaN


# 키워드 추출 단계

In [43]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['axes.unicode_minus'] = False

import warnings 
warnings.filterwarnings('ignore')

from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for idx,doc in enumerate(top7_topic.content) :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
    #print(preprocessed_docs[idx])


사진 넥슨 넥슨 대표 이정헌 서브 브랜드 민트 로켓 신작 데이브 다이버 이하 데이브 개발자 영상 공개 올해 월 정식 출시 예고 개발자 영상 황재호 디렉터 우 찬희 기획 리더 출연 정식 출시 일정 정식 버전 콘텐츠 월 업데이트 소식 전했 데이브 다가오 월 스팀 통해 버전 정식 출시 이후 다양 플랫폼 정식 서비스 추가 계획 이 정식 버전 인족 마을 벨루가 타 쉽 이동 수 있 어두운 분위기 새로운 지역 빙하 통로 만나 볼 수 있 캐릭터 요리 대결 각종 미니게임 캐릭터 스토리 미션 신규 어종 새롭 추가 월 일 콘텐츠 추가 개선 업데이트 실시 맥 운영 체제 게임 플레이 가능 독일어 스페인어 중국어 번 체 추가 지원 포토스 팟 촬영 기능 추가 양식장 밀집 개선 아이템 선택 획득 등 다양 콘텐츠 개선 넥슨 봄 맞 월 일 일 스팀 할인 행사 실시 기존 얼리 엑세스 가격 할인 만 천 원 데이브 구매 수 있 이후 공식 커뮤니티 통해 팬 아트 이벤트 진행 예정 이 데이브 블루홀 탐사 해양 생물 포획 어드벤처 포획 해양 생물 밥집 운영 타이쿤 결합 하이브리드 어드벤처 게임 이 지난 월 스팀 얼리 액세스 진행 글로벌 용자 압도 긍정 평가 이 오 있 문영수 기자 네이버 채널 아이 뉴스 구독 주 재밌 아이 뉴스 영상 보 아이 뉴스 가 아이 뉴스 무단 전재 배포 금지


예약자 수 만 돌파 아키에이지 워 카카오 게임즈 카카오 게임즈 엑스 게임즈 개발 중 인 다중접속 역할 수행 게임 아키에이지 워 사전 예약자 수 만 명 달성 일 밝혔 아키에이지 워 월 일 사전 예약 시작 지 일 만 예약자 만 명 모집 데 이 오 일 출시 앞두 사전 예약자 만 명 달성 아키에이지 워 원작 아키에이지 지식 재산 활용 모바일 크로스 플랫 폼 원작 대비 짙 진 전쟁 전투 요소 특징 이 오픈 월드 배경 필드 전 규모 상전 등 강점 내세웠 카카오 게임즈 사전 예약 누적 인원수 달성 따라 직업 선택 선박 건조대 상자 등 다양 보상 제공 친구 초대 이벤트 참여 이용자 대상 추가 보상 지급 여기 마켓 사전 예약 참여 이용자

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

['사진', '넥슨', '대표', '이정헌', '서브', '브랜드', '민트', '로켓', '신작', '데이브']

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [34]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [49]:
for i in range(len(preprocessed_docs)):
  doc = preprocessed_docs[i] # 핵심키워드 추출할 문서 조회
  feature_names = count_vectorizer.get_feature_names_out() # TF-IDF 단어 목록
  tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
  sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬
  
  # 사용자가 지정한 갯수만큼 키워드 추출
  keywords = extract_keywords(feature_names, sorted_keywords, 5)
  
  top7_topic['keywords'][i]= [k[0] for k in keywords]
  # print("\n===== 원문 =====")
  # print(top7_topic.content[0])
  #print("\n=== 핵심키워드 ===")
  # for k in keywords:
  #     print(k[0])
  

# 기사요약 단계

In [51]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    mecab = Mecab()
    # 각 문장을 소문자로 변환
    sentence1 = [word for word in mecab.pos(sentence1) if word[1][0] in ['N', 'V']]
    sentence2 = [word for word in mecab.pos(sentence2) if word[1][0] in ['N', 'V']]

    union = set(sentence1).union(set(sentence2))
    intersection = set(sentence1).intersection(set(sentence2))
    return len(intersection)/len(union) 


0.5

In [54]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def buildMatrix(sentences):
    score = np.ones(len(sentences) ,dtype=np.float32) # zeros도 가능, 결국 나한테 들어오는 엣지 가중치들을 통해 최초 점수를 가지게 되므로
    # 문장별로 그래프 edge를 Matrix 형태로 생성
    weighted_edge = np.zeros((len(sentences), len(sentences))
                             ,dtype=np.float32)

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j: continue
            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
    
    # normalize 
    for i in range(len(weighted_edge)):
        score[i] = weighted_edge[i].sum()
        weighted_edge[i] /= score[i]

    return score, weighted_edge


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


(array([1.4666667 , 1.1       , 0.16666667, 1.4000001 ], dtype=float32),
 array([[0.        , 0.3409091 , 0.11363637, 0.54545456],
        [0.45454544, 0.        , 0.        , 0.54545456],
        [1.        , 0.        , 0.        , 0.        ],
        [0.57142854, 0.4285714 , 0.        , 0.        ]], dtype=float32))

In [55]:
def scoring(A, P, eps=0.0001, d=0.85, max_iter = 50):
    for iter in range(0,max_iter):
        newP = (1 - d) + d * A.T.dot(P)

        if abs((newP - P).sum()) <= eps:
            return newP
        P = newP
    return newP

In [56]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
    text = sent_tokenize(text)
    score_init, weighted_edge = buildMatrix(text) 
    score = scoring(weighted_edge, score_init)
    
    sorted_scores = sorted(enumerate(score), key=lambda item: item[1], reverse=True)[:n]
    return [(text[s[0]], s[1]) for s in sorted_scores ]

In [83]:
for i in range(len(top7_topic)):
  summary = summarize(top7_topic.content[i], 3)
  top7_topic.summary[i]= [sent[0] for sent in summary]



In [86]:
top7_topic.summary[0]

['데이브는 다가오는 6월 스팀을 통해 PC 버전을 정식 출시하며, 이후 다양한 플랫폼에서도 정식 서비스를 추가할 계획이다.',
 '3월 14일에는 콘텐츠 추가, 개선 업데이트를 실시한다.',
 '개발자 영상에는 황재호 디렉터와 우찬희 기획리더가 출연해 정식 출시 일정과 정식 버전 콘텐츠, 3월 업데이트 소식을 전했다.']

# LDA: gensim

## tokenization

In [ ]:
import konlpy
import re

def tokenize_korean_text(text):
    text = re.sub(r'[▲]','', text) 
    text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))
    text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)
    
    words = []
    stopwords = ['하는', '있다', '있는', '위해', '통해', '한다', '때문'] # 불용어 사전  
    for word, pos in Okt_morphs:
        if pos in ['Adjective', 'Verb', 'Noun', 'Alpha', 'Number'] and word not in stopwords:  # 이 경우에는 형용사, 동사, 명사, 영어, 숫자만 남김
            words.append(word)
    ## word를 이어붙인 string 형태가 아닌 word의 list를 return해주는 게 sklearn lda 준비 과정과의 차이
            
    return words


tokenized_list = []

for text in news['content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

16214
['정부', '올해', '중', '우리나라', '우주', '항공', '분야', '연구개발', 'R', 'D', '전담', '할', '우주', '항공', '청', '출범', '시킬', '계획', '외부', '전문가', '채용', '비중', '제한', '없애고', '우수', '인력', '유치하고자', '주식', '신탁', '예외', '허용', '등', '우주', '항공', '청', '전문성', '높이기', '위', '다양한', '특례', '마련', '된다', '과학기술', '정보통신부', '우주', '항공', '청', '설립', '근거', '되는', '우주', '항공', '청', '설치', '및', '운영', '관', '특별법', '정안', '2일', '입법', '예고', '한다고', '밝혔다', '특별법', '우주', '항공', '분야', '정책', 'R', 'D', '산업', '육성', '등', '총괄', '중앙', '행정기관', '우주', '항공', '청', '설치', '전문', '유연한', '조직', '운영', '하기', '위', '원칙', '기능', '특례', '등', '명시', '먼저', '우주', '항공', '정책', '거버넌스', '구축', '하기', '과', '정통부', '산업', '통상', '자원부', '등', '부처', '개별', '수행', '하던', '우주', '항공', '관련', '기술', '개발', '산업', '육성', '지원', '인재', '양성', '및', '우주', '위험', '대비', '등', '기능', '모두', '우주', '항공', '청', '일원', '화했다', '이를', '기존', '과', '정통부', '장관', '산지', '부', '장관', '등', '소관', '하던', '우주', '항공', '관련', '법률', '우주개발', '진흥', '법', '항공우주', '산업', '촉진법', '문법', '등', '우주', '항공', '청장', '담당', '하도록', '부칙', '개정', '할', '방침', '국가', '우

In [ ]:
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus)) < 21:   # corpus 자체가 list 형태인게 sklearn lda 준비 과정과의 차이
        news.drop(index, axis='index', inplace=True)
        drop_corpus.append(corpus)
    
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

news.reset_index(drop=True, inplace=True)

## vectorization & LDA

In [ ]:
# bigram 생성에 필요한 library
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# vectorize & lda에 필요한 library
from gensim import corpora
from gensim.models.ldamodel import LdaModel

In [ ]:
# Build the bigram models
bigram = Phrases(tokenized_list, min_count=10, threshold=20)
bigram_mod = Phraser(bigram)
 
# See example
print(bigram_mod[tokenized_list[0]])

['정부', '올해', '중', '우리나라', '우주_항공', '분야', '연구개발_R', 'D', '전담', '할', '우주_항공', '청', '출범', '시킬', '계획', '외부_전문가', '채용', '비중', '제한', '없애고', '우수_인력', '유치하고자', '주식_신탁', '예외_허용', '등', '우주_항공', '청', '전문성', '높이기_위', '다양한', '특례', '마련', '된다', '과학기술_정보통신부', '우주_항공', '청_설립', '근거', '되는', '우주_항공', '청_설치', '및', '운영', '관_특별법', '정안', '2일', '입법_예고', '한다고_밝혔다', '특별법_우주', '항공', '분야', '정책', 'R_D', '산업_육성', '등', '총괄', '중앙_행정기관', '우주_항공', '청_설치', '전문', '유연한_조직', '운영', '하기_위', '원칙', '기능', '특례', '등', '명시', '먼저', '우주_항공', '정책', '거버넌스', '구축', '하기', '과_정통부', '산업_통상', '자원부', '등', '부처', '개별', '수행', '하던', '우주_항공', '관련', '기술', '개발', '산업_육성', '지원', '인재_양성', '및', '우주', '위험', '대비', '등', '기능', '모두', '우주_항공', '청', '일원', '화했다', '이를', '기존', '과_정통부', '장관', '산지', '부_장관', '등', '소관', '하던', '우주_항공', '관련', '법률', '우주개발_진흥', '법', '항공우주_산업', '촉진법', '문법', '등', '우주_항공', '청장', '담당', '하도록', '부칙', '개정', '할', '방침', '국가', '우주', '위원회_위원장', '국무총리_대통령', '격상', '해', '위상', '기능', '강화한다', '우주_항공', '청장', '국가', '우주', '위', '새롭게', '위원', '추가', '해', '실무', 

In [ ]:
# tokenized_list의 모든 문서에 대해 bigram을 생성해줌
words_bigram = [bigram_mod[doc] for doc in tokenized_list]

In [ ]:
dictionary = corpora.Dictionary(words_bigram)  # 각 단어에 번호를 할당해줌
    # bigram 포함하는 과정을 생략하고 싶으면, 그냥 바로 여기에 tokenized_list를 넣어주면 됨

dictionary.filter_extremes(no_below=20, no_above=0.80)   
    # 20개의 문서 미만으로 등장하는 단어는 제외, 전체의 80% 이상으로 자주 등장하는 단어는 제외

corpus = [dictionary.doc2bow(text) for text in words_bigram]
print(corpus[0]) # 첫번째 corpus를 테스트로 출력: 몇 번째 단어가 몇 번 나왔는지 저장되어 있음

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 2), (16, 2), (17, 1), (18, 2), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 4), (68, 17), (69, 3), (70, 2), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 3), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 7), (100, 2), (101, 2), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 2)

In [ ]:
ldamodel = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=20, iterations=500)  # 토픽 수: 6

ldamodel.print_topics(num_words=7)  # num_words=10이 default

[(0,
  '0.027*"기술" + 0.014*"기업" + 0.012*"산업" + 0.011*"개발" + 0.011*"지원" + 0.010*"사업" + 0.010*"해"'),
 (1,
  '0.034*"KT" + 0.012*"대표" + 0.009*"카카오" + 0.009*"경영" + 0.009*"는" + 0.008*"전" + 0.008*"기업"'),
 (2,
  '0.025*"서비스" + 0.015*"할_수" + 0.015*"고객" + 0.014*"데이터" + 0.011*"제공" + 0.010*"해" + 0.008*"관리"'),
 (3,
  '0.062*"AI" + 0.021*"챗_GPT" + 0.019*"기술" + 0.016*"를" + 0.015*"인공_지능" + 0.014*"는" + 0.012*"가"'),
 (4,
  '0.022*"것" + 0.011*"이" + 0.008*"그" + 0.007*"고" + 0.006*"미국" + 0.006*"수" + 0.006*"할"'),
 (5,
  '0.019*"서비스" + 0.016*"기업" + 0.016*"플랫폼" + 0.015*"대표" + 0.014*"사업" + 0.012*"글로벌" + 0.011*"콘텐츠"'),
 (6,
  '0.012*"것" + 0.008*"이" + 0.007*"해" + 0.006*"할_수" + 0.006*"입니다" + 0.005*"수" + 0.005*"된"'),
 (7,
  '0.023*"게임" + 0.010*"해" + 0.009*"된" + 0.008*"블록_체인" + 0.008*"것" + 0.007*"거래" + 0.007*"가상_자산"'),
 (8,
  '0.014*"시장" + 0.011*"지난해" + 0.010*"것" + 0.009*"투자" + 0.009*"매출" + 0.007*"증가" + 0.007*"미국"'),
 (9,
  '0.013*"제품" + 0.009*"전자" + 0.008*"를" + 0.008*"해" + 0.008*"할_수" + 0.008*"는" + 0.008*"의"')]

## LDA 시각화

In [ ]:
import pyLDAvis.gensim  # gensim의 ldamodel에 최적화된 라이브러리

vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## 문서별 토픽 할당

In [ ]:
doc_per_topic_list = []

for n in range(len(corpus)):
    doc_topic = ldamodel[corpus[n]]
    doc_topic = sorted(doc_topic, key=lambda x: (x[1]), reverse=True)
    topic_most_pr = doc_topic[0][0]
    topic_pr = doc_topic[0][1]
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

# 실제 review 내용과 join
doc_topic_df = doc_topic_df.join(news)  

doc_topic_df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Doc_Num,Topic,Percentage,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,0,0,0.476281,346630,"정부, '우주항공청' 연내 개청…백지신탁 예외 등 특혜 쏟는다",NaN,\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 ...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
1,1,0,0.405812,346633,우주항공청 전문가 영입 `연봉 10억·외국인 허용`,이준기,\n '한국형 NASA(미 항공우주국)'로 출범하는 우주항공청이 최고의 민간 전문가...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
2,2,8,0.601316,347804,"박정호의 투자유치 '매직'…""SK쉴더스 IPO보다 더 이득""",선한결,"\n ""SK쉴더스의 기업공개(IPO)를 철회해서 아쉬웠는데, IPO보다 더 높은 가...",2023-03-01 18:03:05,NaN,IT/과학,IT 일반,네이버
3,3,3,0.481386,347806,글·그림·로고 제작·번역...인공지능에 다 맡겨 보니,NaN,\n 최근 텍스트 생성 인공지능(AI) 챗봇 챗GPT(ChatGPT)를 이용해 만든...,2023-03-01 18:02:03,NaN,IT/과학,IT 일반,네이버
4,4,8,0.587245,347807,"LG엔솔-혼다, 美 배터리 공장 '첫삽'…기공식 개최",이한얼,\n LG에너지솔루션은 28일(현지시간) 오하이오 주 파이에트 카운티 제퍼슨빌 인근...,2023-03-01 18:26:01,NaN,IT/과학,IT 일반,네이버
